In [5]:
import pandas as pd

df = pd.read_csv("nba_games.csv", index_col= 0)

df = df.sort_values("date")
df = df.reset_index(drop=True)

del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

def add_target(group):
    group["target"] = group["won"].shift(-1)
    return group

df = df.groupby("team", group_keys=False).apply(add_target)

#df[df["team"] == "WAS"]

df["target"][pd.isnull(df["target"])] = 2
df["target"] = df["target"].astype(int, errors="ignore")

df["won"].value_counts()

df["target"].value_counts()

nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]
valid_columns = df.columns[~df.columns.isin(nulls.index)]

df = df[valid_columns].copy()

from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = RidgeClassifier(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split, n_jobs=1)

removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

sfs.fit(df[selected_columns], df["target"])
predictors = list(selected_columns[sfs.get_support()])

def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

predictions = backtest(df, rr, predictors)

from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])

df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

import numpy as np

def find_team_averages(team):
    # Select only numeric columns for rolling mean calculation
    numeric_cols = team.select_dtypes(include=[np.number])
    rolling = numeric_cols.rolling(10).mean()
    
    # You can either return just the rolling averages or merge them back with non-numeric columns
    # Here's an example of merging back with non-numeric columns if needed
    team_rolling = team[numeric_cols.columns].copy()
    team_rolling.loc[:, :] = rolling
    
    # If you have non-numeric columns that you need to carry forward, you can concatenate them back
    non_numeric_cols = team.select_dtypes(exclude=[np.number])
    return pd.concat([non_numeric_cols, team_rolling], axis=1)

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

df = df.dropna()

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])

full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

selected_columns = full.columns[~full.columns.isin(removed_columns)]
sfs.fit(full[selected_columns], full["target"])

predictors = list(selected_columns[sfs.get_support()])

predictions = backtest(full, rr, predictors)

print(accuracy_score(predictions["actual"], predictions["prediction"]))

predictions_with_game_info = predictions.merge(
    full[['team_x', 'team_opp', 'date', 'season']], 
    left_index=True, 
    right_index=True, 
    how='left'
)

# Select only the columns we want to export and ensure they are in the correct order
final_predictions_df = predictions_with_game_info[['team_x', 'team_opp', 'date', 'season', 'prediction']]

# Export the DataFrame to a CSV file
final_predictions_df.to_csv("predictions_with_game_info.csv", index=False)


# selected_columns.to_csv("games.csv", index=False)
#predictions.to_csv("numbers.csv", index=False)

C:\Users\carls\AppData\Local\Temp\ipykernel_10648\1014973525.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"][pd.isnull(df["target"])] = 2


0.5653029110936271


In [13]:
def predict_future_game_outcome(model, team1, team2, full_data, scaler):
    # Assuming 'full_data' has been preprocessed similarly to the training data
    # and contains all necessary features already scaled and selected
    
    # Extract the last row of data for both teams
    team1_last_game = full_data[(full_data['team_x'] == team1)].iloc[-1]
    team2_last_game = full_data[(full_data['team_opp'] == team2)].iloc[-1]
    
    # Ensure we have 134 features for each team
    if len(team1_last_game) != 134 or len(team2_last_game) != 134:
        raise ValueError(f"Each team's data should have 134 features.")

    # Since the model was trained on the differences between the features of the two teams,
    # we will calculate the difference here as well
    game_features = team1_last_game.values - team2_last_game.values
    
    # Reshape the game features to match the input shape expected by the scaler
    game_features = game_features.reshape(1, -1)
    
    # Scale the features using the trained scaler
    game_features_scaled = scaler.transform(game_features)
    
    # Predict the outcome with the trained model
    predicted_outcome = model.predict(game_features_scaled)
    
    return predicted_outcome[0]

# Example usage
team1 = 'LAL'  # Los Angeles Lakers
team2 = 'BOS'  # Boston Celtics

# Predict the outcome
predicted_outcome = predict_future_game_outcome(rr, team1, team2, full, scaler)
print(f"The model predicts a {predicted_outcome} for {team1} against {team2}.")


ValueError: Expected 134 features, but got 30 features for team 1 and 30 features for team 2.